In [3]:
import numpy as np
import pandas as pd

In [59]:
state2abbr = {"Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA",
    "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI",
    "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS", "Kentucky": "KY",
    "Louisiana": "LA", "Maine": "ME", "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN",
    "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV", "New Hampshire": "NH",
    "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY", "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH",
    "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC", "South Dakota": "SD",
    "Tennessee": "TN", "Texas": "TX", "Utah": "UT", "Vermont": "VT", "Virginia": "VA", "Washington": "WA", "West Virginia": "WV",
    "Wisconsin": "WI", "Wyoming": "WY", "District of Columbia": "DC",
}
    
# invert the dictionary
abbr2state = dict(map(reversed, state2abbr.items()))
state_list = list(state2abbr.keys())
state_list.pop(1)

'Alaska'

In [78]:
def weight_average_column(df, col_name):
    data = df[["State", "County", "TotalPop", col_name]]
    data["tmp"] = data["TotalPop"] * data[col_name]
    agg_data = data.groupby(["State", "County"], as_index = False).agg("sum")
    agg_data[col_name] = agg_data["tmp"] / agg_data["TotalPop"]
    return agg_data[["State", "County", col_name]]

def load_clean_census_data():
    census_var_list = ["State", "County", "TotalPop", "Men", "Women"]
    census_weight_list = ["Hispanic", "White", "Black", "Native", "Asian", "Pacific", "Drive", "Carpool", "Transit", "Walk", "OtherTransp", "WorkAtHome", "MeanCommute"]
    full_data = pd.read_csv("../datasets/acs2017_census_tract_data.csv")
    full_data = full_data.dropna(how = "all")
    full_data = full_data[census_var_list + census_weight_list]
    basic_df = full_data[census_var_list]
    agg_data = basic_df.groupby(["State", "County"], as_index = False).agg("sum")
    for weight_name in census_weight_list:
        column = weight_average_column(full_data, weight_name)
        agg_data = pd.merge(agg_data, column, on=["State", 'County'])
    agg_data = agg_data[agg_data["State"] != 'Puerto Rico']
    agg_data["County"] = agg_data["County"].apply(lambda s: s.split(" ")[0] if len(s.split(" ")) == 2 else s.split(" ")[0] + " " + s.split(" ")[1])
    return agg_data

In [96]:
census_df = load_clean_census_data()
census_df

/home/fanshi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute
0,Alabama,Autauga,55036,26899,28137,2.672443,75.422703,18.932544,0.322632,0.939085,0.024602,85.874886,9.538171,0.060964,0.586184,1.379277,2.543982,25.899355
1,Alabama,Baldwin,203360,99527,103833,4.445085,83.080754,9.462884,0.752043,0.682431,0.000000,84.534211,7.777040,0.110775,0.810847,1.161533,5.627419,26.951154
2,Alabama,Barbour,26201,13976,12225,4.202309,45.736300,47.829587,0.175959,0.578573,0.000000,83.243827,11.156055,0.298237,2.365803,1.693775,1.244479,23.968818
3,Alabama,Bibb,22580,12251,10329,2.337236,74.634783,21.996639,0.382334,0.000000,0.000000,86.367228,9.433627,0.629880,0.314628,1.854570,1.438109,29.867551
4,Alabama,Blount,57667,28490,29177,9.000794,87.358163,1.527107,0.341069,0.153852,0.000000,86.841570,10.123662,0.131966,0.415479,0.370345,2.116228,35.106992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,Wyoming,Sweetwater,44527,22981,21546,16.007211,79.631911,0.774173,0.559919,0.579363,0.461641,77.371013,14.416172,2.582907,2.883904,1.290639,1.458735,20.481366
3216,Wyoming,Teton,22923,12169,10754,14.977054,81.521943,0.463574,0.256825,2.190944,0.000000,68.318008,6.679649,3.934843,11.555333,3.827496,5.696990,14.263569
3217,Wyoming,Uinta,20758,10593,10165,9.069024,87.644397,0.134531,0.940423,0.110964,0.000000,77.370320,14.908642,3.383390,1.098203,1.323225,2.022931,19.983414
3218,Wyoming,Washakie,8253,4118,4135,14.244820,82.212529,0.310384,0.354441,0.126257,0.000000,77.088198,10.307658,0.031564,6.916552,1.359700,4.296329,14.774930


In [83]:
def extract_feature(full_df, feature_name, col_name):
    df = full_df[(full_df["Description"] == feature_name) & (full_df["2018"] != '(NA)')]
    df = df[["GeoFIPS", "GeoName", "2018"]]
    df["2018"] = df["2018"].astype(int)
    df = df.rename(columns={"2018":col_name})
    return df

def load_clean_economic_data():
    raw_data = pd.read_csv("../datasets/economic_profile_raw.csv", encoding='latin-1')
    raw_data = raw_data.dropna(how="all")
    df_data = raw_data[["GeoFIPS", "GeoName", "Description", "Unit", "2018"]]
    df_data = df_data[df_data["GeoFIPS"].str.len() == 8]
    df_data["GeoFIPS"] = df_data["GeoFIPS"].apply(lambda x: x[2:-1])
    df_county = df_data[~df_data["GeoFIPS"].str.endswith("000")]
    county_income_df = extract_feature(df_county, " Per capita personal income 4/", "income")
    county_employments_df = extract_feature(df_county, "Total employment (number of jobs)", "totalEmploy")
    county_salary_df = extract_feature(df_county, " Average wages and salaries", "salary")
    agg_df = county_income_df
    agg_df = pd.merge(agg_df, county_employments_df, on=["GeoFIPS", "GeoName"])
    agg_df = pd.merge(agg_df, county_salary_df, on=["GeoFIPS", "GeoName"])
    agg_df["State"] = agg_df["GeoName"].apply(lambda s: abbr2state[s[-3:-1] if s[-1] == "*" else s[-2:]])
    agg_df["County"] = agg_df["GeoName"].apply(lambda s: s.split(",")[0])
    agg_df["County"] = agg_df["County"].apply(lambda s: s.split(" ")[0] if len(s.split(" ")) == 1 else s.split(" ")[0] + " " + s.split(" ")[1])                                           
    agg_df = agg_df.drop(columns=["GeoName"])
    return agg_df

economic_df = load_clean_economic_data()

In [90]:
join_data = pd.merge(census_df, economic_df, on=["State", "County"])

In [91]:
join_data

,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,...,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,GeoFIPS,income,totalEmploy,salary
0,Alabama,Autauga,55036,26899,28137,2.672443,75.422703,18.932544,0.322632,0.939085,...,9.538171,0.060964,0.586184,1.379277,2.543982,25.899355,01001,42504,18172,38739
1,Alabama,Baldwin,203360,99527,103833,4.445085,83.080754,9.462884,0.752043,0.682431,...,7.777040,0.110775,0.810847,1.161533,5.627419,26.951154,01003,46205,115748,37428
2,Alabama,Barbour,26201,13976,12225,4.202309,45.736300,47.829587,0.175959,0.578573,...,11.156055,0.298237,2.365803,1.693775,1.244479,23.968818,01005,35067,11506,38032
3,Alabama,Bibb,22580,12251,10329,2.337236,74.634783,21.996639,0.382334,0.000000,...,9.433627,0.629880,0.314628,1.854570,1.438109,29.867551,01007,30559,6454,40495
4,Alabama,Blount,57667,28490,29177,9.000794,87.358163,1.527107,0.341069,0.153852,...,10.123662,0.131966,0.415479,0.370345,2.116228,35.106992,01009,35671,15702,35305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3079,Wyoming,Sweetwater,44527,22981,21546,16.007211,79.631911,0.774173,0.559919,0.579363,...,14.416172,2.582907,2.883904,1.290639,1.458735,20.481366,56037,55352,27908,60194
3080,Wyoming,Teton,22923,12169,10754,14.977054,81.521943,0.463574,0.256825,2.190944,...,6.679649,3.934843,11.555333,3.827496,5.696990,14.263569,56039,230141,33629,50939
3081,Wyoming,Uinta,20758,10593,10165,9.069024,87.644397,0.134531,0.940423,0.110964,...,14.908642,3.383390,1.098203,1.323225,2.022931,19.983414,56041,40425,11806,42424
3082,Wyoming,Washakie,8253,4118,4135,14.244820,82.212529,0.310384,0.354441,0.126257,...,10.307658,0.031564,6.916552,1.359700,4.296329,14.774930,56043,49310,5255,42110


In [92]:
climate_df = pd.read_csv("../datasets/climate/cleaned/climate.csv")

In [95]:
climate_df[climate_df["Year"] == 2018]

,Unnamed: 0,State,County,Temp Avg,Year,Temp Max,Temp Min,Precipitation,Location
0,0,AK,Aleutians East Borough,38.8,2018,44.1,33.6,41.35,"Aleutians East Borough, AK"
3,3,AK,Aleutians West Census Area,39.7,2018,43.6,35.7,47.58,"Aleutians West Census Area, AK"
6,6,AK,Anchorage Municipality,32.9,2018,39.4,26.3,64.83,"Anchorage Municipality, AK"
9,9,AK,Bethel Census Area,33.1,2018,40.7,25.4,26.09,"Bethel Census Area, AK"
12,12,AK,Bristol Bay Borough,37.2,2018,45.2,29.2,22.72,"Bristol Bay Borough, AK"
...,...,...,...,...,...,...,...,...,...
9393,9393,WY,Sweetwater County,44.4,2018,57.6,31.2,9.97,"Sweetwater County, WY"
9396,9396,WY,Teton County,37.3,2018,48.7,25.8,44.66,"Teton County, WY"
9399,9399,WY,Uinta County,42.8,2018,56.0,29.7,12.31,"Uinta County, WY"
9402,9402,WY,Washakie County,46.3,2018,58.6,33.9,15.11,"Washakie County, WY"
